In [ ]:
# pymortar 1.0.5
class Client:
    """Method to create a new Pymortar client

    The configuration takes the following (optional) parameters:

    - mortar_address: address + port to connect to, e.g. "localhost:9001". Defaults to $MORTAR_API_ADDRESS from the environment. Currently expects a TLS-secured endpoint
    - username: your Mortar API username. Defaults to MORTAR_API_USERNAME env var
    - password: your Mortar API password. Defaults to MORTAR_API_PASSWORD env var

    Keyword Args:
        cfg (dict or None): configuration dictionary. Takes the following (optional) keys:

    Returns:
        client (Client): PyMortar client
    """
    def __init__(self, cfg=None):
        if cfg is not None:
            self._cfg = cfg
        else:
            self._cfg = {}

        # get username/password from environment or config file
        if 'username' not in self._cfg or not self._cfg['username']:
            self._cfg['username'] = os.environ.get('MORTAR_API_USERNAME')
        if 'password' not in self._cfg or not self._cfg['password']:
            self._cfg['password'] = os.environ.get('MORTAR_API_PASSWORD')

        if self._cfg.get('mortar_address') is None:
            self._mortar_address = os.environ.get('MORTAR_API_ADDRESS','mortardata.org:9001')
        else:
            self._mortar_address = self._cfg.get('mortar_address')

        self._connect()

        def connectivity_event_callback(event):
            if event == grpc.ChannelConnectivity.TRANSIENT_FAILURE:
                logging.error("Transient failure detected; reconnecting in 10 sec")
                time.sleep(10)
                self._connect()
            else:
                logging.info("Got event {0}".format(event))

        # listen to channel events
        self._channel.subscribe(connectivity_event_callback)

        if os.path.exists(".pymortartoken.json"):
            self._token = json.load(open(".pymortartoken.json", "r"))
            logging.info("loaded .pymortartoken.json token")
        else:
            self._token = None

        # TODO: handle the refresh token recycling automatically for the user
        # TODO: break this out into a method that can be called when we notice that the token is expired

        # FIX: comment this out to allow token to expire and see connection state
        if self._token is None:
            self._refresh()
